Dictionary by hand:
0 - Magnetom Verio, SIEMENS, 3 T
1 - Magnetom Avanto, SIEMENS, 1,5 Т 
    Siemens MAGNETOM Avanto, 1,5T
2 - Magnetom Symphony, SIEMENS, 1.5 Т
3 - Panorama, PHILIPS, 1.0 Т
4 - None

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('/notebook/MS_trainset_anon_csv/reports.csv', sep = '|')
data.head()

,id,text,conclusion,label
0,01_414,МАГНИТНО-РЕЗОНАНСНАЯ ТОМОГРАФИЯ ГОЛОВНОГО МОЗ...,ЗАКЛЮЧЕНИЕ: МРТ-данные соответствуют изменени...,ms
1,01_344,МАГНИТНО-РЕЗОНАНСНАЯ ТОМОГРАФИЯ ГОЛОВНОГО МОЗ...,"ЗАКЛЮЧЕНИЕ: МРТ данные, вероятнее всего, соотв...",ms
2,01_316,МАГНИТНО-РЕЗОНАНСНАЯ ТОМОГРАФИЯ ГОЛОВНОГО МОЗГ...,"ЗАКЛЮЧЕНИЕ: МРТ-данные соответствуют, вероятне...",ms
3,01_294,МАГНИТНО-РЕЗОНАНСНАЯ ТОМОГРАФИЯ ГОЛОВНОГО МОЗГ...,ЗАКЛЮЧЕНИЕ: МРТ-данные соответствуют демиелин...,ms
4,01_196,МАГНИТНО-РЕЗОНАНСНАЯ ТОМОГРАФИЯ ГОЛОВНОГО МОЗГ...,"ЗАКЛЮЧЕНИЕ: МРТ данные, вероятнее всего, соотв...",ms


In [7]:
data.label.value_counts()

ms       147
none     130
other     20
Name: label, dtype: int64

In [12]:
data.shape

(297, 5)

In [103]:
data['label'] = data.apply(lambda row: 1 if row.label == 'ms' else 0, axis = 1)

In [13]:
data['model'] = data.apply(lambda row: row.text[row.text.find('(')+1 : row.text.find(')')], axis = 1) 

In [39]:
data.model.value_counts()

Magnetom Verio, SIEMENS, 3 T         148
Magnetom Avanto, SIEMENS, 1,5 Т       76
Magnetom Avanto, SIEMENS, 1.5 Т       30
Magnetom Avanto, SIEMENS, 1.5 T       25
Siemens MAGNETOM Avanto, 1,5T          4
Magnetom Symphony, SIEMENS, 1.5 Т      4
Magnetom Symphony, SIEMENS, 1,5 Т      3
Panorama, PHILIPS, 1.0 Т               2
Magnetom Avanto, SIEMENS 1,5 T         2
Гадовист 7,5                           1
Magnetom Avanto, SIEMENS, 1,5 T        1
Magnetom Verio, SIEMENS, 3 Т           1
Name: model, dtype: int64

In [14]:
model_code = []
for i in data['model']:
    if i == 'Magnetom Verio, SIEMENS, 3 T' or i == 'Magnetom Verio, SIEMENS, 3 Т':
        model_code.append(0)
    elif i == 'Magnetom Avanto, SIEMENS, 1,5 Т' or i == 'Magnetom Avanto, SIEMENS, 1.5 Т' or i == 'Magnetom Avanto, SIEMENS, 1.5 T' or i == 'Magnetom Avanto, SIEMENS 1,5 T' or i == 'Magnetom Avanto, SIEMENS, 1,5 T' or i == 'Siemens MAGNETOM Avanto, 1,5T':
        model_code.append(1)
    elif i == 'Magnetom Symphony, SIEMENS, 1.5 Т' or i == 'Magnetom Symphony, SIEMENS, 1,5 Т':
        model_code.append(2)
    elif i == 'Panorama, PHILIPS, 1.0 Т':
        model_code.append(3)
    elif i == 'Гадовист 7,5':
        model_code.append(4)

In [15]:
data['model_code'] = model_code

In [16]:
data['model_code'].value_counts()

0    149
1    138
2      7
3      2
4      1
Name: model_code, dtype: int64

In [82]:
concl_text_ms, concl_text_other, concl_text_none = '', '', ''
for i in data['text'][data.label == 'ms'].index:
    concl_text_ms += data['conclusion'][i]

for i in data['text'][data.label == 'none'].index:
    concl_text_none += data['conclusion'][i]

for i in data['text'][data.label == 'other'].index:
    concl_text_other += data['conclusion'][i]

In [59]:
from rutermextract import TermExtractor

In [67]:
term_extractor = TermExtractor()

In [84]:
for term in term_extractor(concl_text_ms, limit = 10):
    print (term.normalized, term.count)

заключение 147
демиелинизирующий процесс 141
головной мозг 100
большое мозг 12
спинной мозг 10
оба полушариях 7
левый полушарие 5
демиелинизирующий характер 5
признаки 5
венозная ангиома 4


In [85]:
for term in term_extractor(concl_text_none, limit = 10):
    print (term.normalized, term.count)

исследование 131
очаговая патология 130
головной мозг 130
заключение 130
признаки 3
турецкий седло 2
выявленные изменение 2
выраженная вентрикуломегалия 2
внутренняя гидроцефалия 2
киста 2


In [86]:
for term in term_extractor(concl_text_other, limit = 10):
    print (term.normalized, term.count)

заключение 19
большое мозг 8
мрт-исследование 6
сосудистый генез 5
головной мозг 5
оба полушариях 4
очаги 4
исследование 4
право 3
полушарие 3


In [3]:
import string
punc = str.maketrans({key: None for key in '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~”„–'})

In [4]:
data['splt'] = data.apply(lambda row: row.conclusion.translate(punc).lower().split(),  axis=1) 

In [24]:
data = data[['id', 'label', 'splt', 'model_code']]

In [25]:
data.head()

,id,label,splt,model_code
0,01_414,ms,"[заключение, мртданные, соответствуют, изменен...",1
1,01_344,ms,"[заключение, мрт, данные, вероятнее, всего, со...",1
2,01_316,ms,"[заключение, мртданные, соответствуют, вероятн...",1
3,01_294,ms,"[заключение, мртданные, соответствуют, демиели...",1
4,01_196,ms,"[заключение, мрт, данные, вероятнее, всего, со...",1


In [6]:
def alphas(row):
    tmp = []
    for i in row:
        if str.isalpha(i):
            tmp.append(i)
    return tmp

In [7]:
data['splt'] = data.apply(lambda row: alphas(row.splt), axis = 1)

In [116]:
data.to_csv('/notebook/MS_trainset_anon_csv/reports_mode.csv', sep=',', header=True, index=False, encoding='utf-8')

In [26]:
import gensim
from nltk import word_tokenize, FreqDist

In [27]:
tokens = []
for i in range(data.shape[0]):
    for j in data.splt[i]:
        tokens.append(j)

In [28]:
fdist = FreqDist(tokens)

In [29]:
len(fdist)

315

In [ ]:
def one_hot_encoder(data, fdist):
    for i in fdist.keys():
        data[i] = data.apply(lam)
    
    

In [35]:
fdist.keys()

dict_keys(['заключение', 'мртданные', 'соответствуют', 'изменениям', 'в', 'обоих', 'полушариях', 'головного', 'мозга', 'вероятнее', 'всего', 'демиелинизирующего', 'характера', 'мрт', 'данные', 'демиелинизирующему', 'процессу', 'и', 'спинного', 'венозная', 'ангиома', 'левом', 'полушарии', 'большого', 'очаговому', 'поражению', 'вещества', 'не', 'совсем', 'ясного', 'генеза', 'могут', 'соответствовать', 'признаки', 'арахноидальной', 'кисты', 'левой', 'височной', 'области', 'нельзя', 'исключить', 'вариант', 'бало', 'небольшой', 'на', 'конвекситальной', 'поверхности', 'правой', 'лобной', 'доли', 'киста', 'шишковидной', 'железы', 'очагу', 'правом', 'более', 'вероятно', 'характерному', 'для', 'процесса', 'смешанной', 'гидроцефалии', 'мртпризнаки', 'пансинусита', 'очаговым', 'полушарий', 'с', 'учетом', 'изменений', 'веществе', 'выявленные', 'особенности', 'строения', 'гипофиза', 'варианту', 'или', 'быть', 'косвенным', 'признаком', 'микроаденомы', 'целесообразна', 'консультация', 'эндокринолога'

In [105]:
data['full_text'] = data.apply(lambda row: row.text + row.conclusion, axis = 1)

In [106]:
ms = data.full_text[(data.label == 1)].values
none = data.full_text[(data.label == 0)].values

In [107]:
texts = np.concatenate((ms, none), axis=0)
labels = [1] * len(ms) + [0] * len(none)

In [110]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

In [111]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(texts)
X_train_counts.shape

(297, 1395)

In [115]:
ngrams = [(2,2), (3,3), (1,3)]
clf = LogisticRegression(random_state = 2)

for i in ngrams:
    count_vect = CountVectorizer(ngram_range = i)
    X_train_counts = count_vect.fit_transform(texts)
    scores = cross_val_score(clf, X_train_counts, labels, scoring = 'f1', cv = 10)
    print (round(scores.mean(),2))

/root/.pyenv/versions/3.6.7/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


1.0


/root/.pyenv/versions/3.6.7/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


1.0


/root/.pyenv/versions/3.6.7/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


1.0
